# Корреляции

#### Импорты

In [119]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.metrics import matthews_corrcoef
from statsmodels.sandbox.stats.multicomp import multipletests 
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

In [120]:
def dataset_cols_corr (data):
    corr_data=[]
    for i, lhs_column in enumerate(data.columns):
        for j, rhs_column in enumerate(data.columns):
            if i >= j:
                continue
        
            corr, p = stats.wilcoxon(data[lhs_column], data[rhs_column])
            corr_data.append([lhs_column, rhs_column, corr, p])
    return corr_data

#### Данные

In [131]:
data = pd.read_csv('AUCs.txt',delimiter='\t')

corr_data = dataset_cols_corr(data.drop(['Unnamed: 0'],axis=1))

df = pd.DataFrame.from_records(corr_data)
df.columns = ['A', 'B', 'corr', 'p']
df.sort_values(by='corr', ascending=True)

,A,B,corr,p
0,C4.5,C4.5+m,6.5,0.010757
5,C4.5+cf,C4.5+m+cf,10.0,0.022909
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
1,C4.5,C4.5+cf,43.0,0.861262


#### Корреляция Пирсона 
Мера линейной зависимости между 2 случайными величинами. Неустойчива к выбросам.

In [135]:
#data.corr()
pearsonr(data['C4.5'],data['C4.5+m'])[0]

0.9918785225254219

#### Корреляция Спирмена 
Мера силы монотонной взаимосвязи = кор. Пирсона между рангами наблюдений. Более устойчив к выбросам.

In [136]:
spearmanr(data['C4.5'],data['C4.5+m'])[0]

0.9956043956043955

#### Корреляция Мэтьюса
... бинарными переменными. Для сравнения меток и предсказаний:

In [140]:
matthews_corrcoef([0,1,1,0],[1,1,1,0])

0.5773502691896258

Для проверки гипотез:

In [52]:
# 203 из 718 + 239 из 515
a = 718
b = 203
c = 515
d = 239
mcc = (a*d - b*c)/math.sqrt((a+b)*(a+c)*(b+d)*(c+d))
round(mcc, 3)

0.109

#### Коэффициент V Крамера 
... категориальными переменными. Для расчета нужно сформировать таблицу сопряженности и посчитать значение статистики Хи квадрат:

In [75]:
contingencyTable = np.array([[197, 111, 33], [382, 685, 331], [110, 342, 333]])
hi_square_koef = chi2_contingency(contingencyTable)[0]
hi_square_koef

293.68311039689746

А потом вычислить сам коэффициент:

In [76]:
n = sum([sum(x) for x in contingencyTable])
kramer_v = np.sqrt(chi2_contingency(contingencyTable)[0] / (n * 2))
kramer_v

0.2412013934500338

#### FWER. Поправка Бонферрони
Для множественной проверки гипотез. Вероятность совершить хотя бы 1 ошибку 1 рода. Достигаемый уровень значимости всех гипотез сравнивается с alpha / m. Если он меньше, то гипотеза отвергается. При использовании данного метода вероятность ошибки 1 рода ограничивается гораздо более низкой величиной, чем α, что увеличивает вероятность ошибки 2 рода.

#### FWER. Метод Холма
Для множественной проверки гипотез. Из достигаемых уровней значимости составляется вариационный ряд. Сравниваем самый маленький уровень с alpha/m, если он больше, то принимается эта гипотеза и все следующие. Если нет, то отвергается и процедура продолжается, но сравнивается уже с alpha / (m-1) и т.д. Данный метод всегда лучше Бонферрони.

In [141]:
reject, p_corrected, a1, a2 = multipletests(df['p'], 
                                            alpha = 0.05, 
                                            method = 'holm') 
reject

array([False, False, False, False, False, False])

#### FDR. Метод Бенджамини-Хохберга
Для множественной проверки гипотез. Повышает точность, мощность, вероятность ошибки 1 рода. Используется вариационный ряд, только сравнения начинаются с другого конца. Выбирается самый большой p-val, если он меньше alpha/m то все отвергается. В ином случае процедура продолжается, но уже c alpa*i/m и тд. Применяется только когда выборки независимы. Используется повсеместно.

In [142]:
reject, p_corrected, a1, a2 = multipletests(df['p'], 
                                            alpha = 0.05, 
                                            method = 'fdr_bh') 
reject

array([ True, False,  True, False, False,  True])

#### Регрессия

Анализ влияния признаков на результат. Для начала прочитаем данные:

In [180]:
botswana_df = pd.read_csv('botswana.tsv', delimiter='\t')
botswana_df.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


Обработаем пропуски

In [181]:
botswana_df['nevermarr'] = np.where(botswana_df['agefm'].isnull(),1,0) 
botswana_df.drop('evermarr',1,inplace=True)
botswana_df['agefm'].fillna(0,inplace=True)
botswana_df.heduc = botswana_df.apply(lambda x: -1 if (x.nevermarr == 1 and pd.isnull(x.heduc)) else x.heduc, axis=1)
botswana_df['idlnchld_noans'] = np.where(botswana_df['idlnchld'].isnull(),1,0) 
botswana_df['heduc_noans'] = np.where(botswana_df['heduc'].isnull(),1,0) 
botswana_df['usemeth_noans'] = np.where(botswana_df['usemeth'].isnull(),1,0) 
botswana_df['idlnchld'].fillna(-1,inplace=True)
botswana_df['heduc'].fillna(-2,inplace=True)
botswana_df['usemeth'].fillna(-1,inplace=True)

Построим модель и проанализируем ее характеристики

In [254]:
model = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans +'\
                    'heduc_noans + usemeth_noans', 
             data=botswana_df)
fitted = model.fit(cov_type='HC1')
print fitted.summary()

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Fri, 07 Sep 2018   Prob (F-statistic):               0.00
Time:                        20:00:49   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

По p-value видно что показатели религии, радио и тв слабо влияют на результат.

In [ ]:
model2 = smf.ols('ceb ~ age + educ  + idlnchld + knowmeth + usemeth +'\
                    'agefm + heduc + urban + electric + bicycle + nevermarr + idlnchld_noans +'\
                    'heduc_noans + usemeth_noans', 
             data=botswana_df)
fitted2 = model2.fit(cov_type='HC1')
print fitted2.summary()

In [ ]:
print('Breusch-Pagan test: p=%s' % sms.het_breushpagan(fitted.resid, fitted.model.exog)[1])

Проверка гипотезы о том, что качество моделей одинаково.

In [ ]:
print "F=%f, p=%f, k1=%f" % fitted.compare_f_test(fitted2)